In [1]:
%%capture
import pyterrier as pt
if not pt.started():
  pt.init()

In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/workspace/corpus.csv')

from xml.etree import ElementTree
tree = ElementTree.parse('/content/drive/MyDrive/workspace/topics.xml')
data = tree.getroot()
titles = []
title_ids = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    title_ids.append(num)
topics = pd.DataFrame(titles,columns = ['query'])
topics['qid'] = topics.index + 51
columns_titles = ["qid","query"]
topics=topics.reindex(columns=columns_titles)

qrels_path=("/content/drive/MyDrive/workspace/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

docnos = qrels.docno


In [3]:
pd_indexer = pt.DFIndexer("./pd_index")
indexref = pd_indexer.index(df["text"], df["docno"])

index = pt.IndexFactory.of(indexref)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res=retr.transform(topics)

res

,qid,docid,docno,rank,score,query
0,51,297468,S885c6b4f-Ad3259e81,0,15.174829,do we need sex education in schools
1,51,330918,S6353b9ac-A9cb2062c,1,14.984042,do we need sex education in schools
2,51,222529,S428a8bea-Aba4c7b80,2,14.776028,do we need sex education in schools
3,51,279241,Sd32774d-A5bbc95,3,14.715350,do we need sex education in schools
4,51,330916,S6353b9ac-A2df6348e,4,14.638091,do we need sex education in schools
...,...,...,...,...,...,...
49995,100,256729,S2ea519c-Aed35ffc8,995,3.581751,do we need cash
49996,100,29220,Sa68e9864-Af79ef325,996,3.580354,do we need cash
49997,100,231260,Sfb30ce24-Aa94950e7,997,3.580354,do we need cash
49998,100,294707,S78819fc5-Ae076e471,998,3.576581,do we need cash


In [7]:
res_copy = res.copy()
# res_copy = res_copy[res_copy['rank']<5]
res_copy = res_copy[res_copy['docno'].isin(docnos)]
res_copy.head(10)

,qid,docid,docno,rank,score,query
0,51,297468,S885c6b4f-Ad3259e81,0,15.174829,do we need sex education in schools
1,51,330918,S6353b9ac-A9cb2062c,1,14.984042,do we need sex education in schools
2,51,222529,S428a8bea-Aba4c7b80,2,14.776028,do we need sex education in schools
5,51,213692,S11b834b9-Aabe96a35,5,14.538698,do we need sex education in schools
6,51,314475,S937b1a96-A198bbc54,6,14.524366,do we need sex education in schools
7,51,239957,S3497c718-A2e6437a0,7,14.516631,do we need sex education in schools
8,51,198593,S471f6d0a-A6a764709,8,14.431860,do we need sex education in schools
14,51,37968,S13f4e445-A8e79d876,14,14.208333,do we need sex education in schools
15,51,182595,S471f6ceb-Ab64777b5,15,14.197628,do we need sex education in schools
16,51,4205,Sf27da082-A442fab1b,16,14.165045,do we need sex education in schools


In [5]:
eval = pt.Utils.evaluate(res_copy,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])
eval

{'bpref': 0.6261477227722415,
 'ndcg_cut_25': 0.6332115060794034,
 'ndcg_cut_5': 0.6118904950108662}

In [6]:
eval = pt.Utils.evaluate(res,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])
eval

{'bpref': 0.6261477227722415,
 'ndcg_cut_25': 0.45749865160257747,
 'ndcg_cut_5': 0.5527540851759042}